In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical, plot_model

In [2]:
dataset = mnist.load_data()
(X_train, y_train), (X_test, y_test) = dataset

print("shape of X =", X_train.shape)
print("shape of y =", y_train.shape)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
shape of X = (60000, 28, 28)
shape of y = (60000,)


In [3]:
# Flatten the inputs
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Convert the target to one-hot encoding
number_of_classes = 10
y_train = to_categorical(y_train, num_classes=number_of_classes)
y_test = to_categorical(y_test, num_classes=number_of_classes)


print("shape of X =", X_train.shape)
print("shape of y =", y_train.shape)

shape of X = (60000, 784)
shape of y = (60000, 10)


# Install the keras-tuner library

In [4]:
!pip install keras-tuner -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.0 MB/s eta 0:00:00


In [5]:
def build_model(hp):
    model = Sequential()
    model.add( Input(shape = (784,)) )
    number_of_layers = hp.Int("num_layers", 1, 3)
    for i in range(number_of_layers):
      units = hp.Int(f"units_{i}", min_value=5, max_value=20, step=5)
      activation = hp.Choice("activation", ["relu", "tanh", "sigmoid"])
      model.add( Dense(units, activation=activation) ) #hidden layer

    model.add( Dense(10, activation='softmax') ) #output layer, activation must be softmax

    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"],
    )
    return model

In [6]:
import keras_tuner

build_model(keras_tuner.HyperParameters())

<Sequential name=sequential, built=True>

In [7]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=1,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

In [8]:
tuner.search_space_summary()


Search space summary
Default search space size: 3
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 5, 'max_value': 20, 'step': 5, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}


In [9]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))


Trial 3 Complete [00h 00m 38s]
val_accuracy: 0.7283999919891357

Best val_accuracy So Far: 0.8586999773979187
Total elapsed time: 00h 02m 02s


In [10]:
tuner.results_summary()


Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
num_layers: 1
units_0: 20
activation: tanh
Score: 0.8586999773979187

Trial 2 summary
Hyperparameters:
num_layers: 2
units_0: 15
activation: relu
units_1: 15
Score: 0.7283999919891357

Trial 1 summary
Hyperparameters:
num_layers: 2
units_0: 20
activation: relu
units_1: 5
Score: 0.6991000175476074
